In [81]:
import numpy as np

In [82]:
import pandas as pd

In [83]:
import matplotlib.pyplot as plt

In [84]:
import re

In [85]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [86]:
from sklearn import svm

In [87]:
def categorize_numerically(df, column_name):
    unique_vals = df[column_name].unique()
    column_vals = df[column_name].astype('category').cat.codes
    df[column_name] = column_vals
    return unique_vals, df

def categorize_quartile(df, column_name, quartile, labels):
    df[column_name] = pd.qcut(df[column_name].values, quartile, labels)
    df[column_name] = df[column_name].astype('category').cat.codes
    return (labels,df)

def categorize_cabin(df, cabin_name):
    column_vals = df[cabin_name].astype(str)
    reg = r"\w\d+"
    regexp = re.compile(reg)
    reg2 = r"\d+"
    regexp2 =  re.compile(reg2)

    sep = []
    for val in column_vals:
        if(val != 'nan'):
            #if the array has more than one value, simplify to the first value UwU

            if( len(regexp.findall(val)) > 1):
                val = regexp.findall(val)[0]
            if(len(val) > 1):
                val = val[0]
            sep.append(val)
        else:
            sep.append(val)
    df[cabin_name] = sep
    unique_vals = df[cabin_name].unique()
    column_vals = df[cabin_name].astype("category").cat.codes
    df[cabin_name] = column_vals
    return (unique_vals, df)

In [88]:
titanic_train = pd.read_csv('train.csv')

In [89]:
titanic_test = pd.read_csv('test.csv')

In [90]:
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [91]:
not_features = ['PassengerId','Name','Ticket']
features = ['Sex', 'Pclass','Age', 'Fare','Embarked', 'Cabin', 'Sibsp', 'Parch']
save_id = titanic_test['PassengerId']
titanic_train = titanic_train.drop(columns = not_features)
titanic_test = titanic_test.drop(columns = not_features)

In [92]:
sex_mapping, titanic_train = categorize_numerically(titanic_train, 'Sex')
sex_mapping, titanic_test = categorize_numerically(titanic_test, 'Sex')

embark_mapping, titanic_train= categorize_numerically(titanic_train, 'Embarked')
embark_mapping, titanic_test= categorize_numerically(titanic_test, 'Embarked')

room_mapping, titanic_train = categorize_cabin(titanic_train, 'Cabin')
room_mapping, titanic_test = categorize_cabin(titanic_test, 'Cabin')

outputs=["very cheap", "cheap","fair", "expensive","very expensive"]
wealth_mapping, titanic_train = categorize_quartile(titanic_train, 'Fare', 5, outputs)
wealth_mapping, titanic_test = categorize_quartile(titanic_test, 'Fare', 5, outputs)

outputs = ["baby", "child", "teenager","young adult","adult","elderly"]
age_mapping, titanic_train = categorize_quartile(titanic_train, 'Age',6,outputs)
age_mapping, titanic_test = categorize_quartile(titanic_test, 'Age',6,outputs)

In [93]:
y = titanic_train['Survived'].values
titanic_train = titanic_train.drop(columns = ['Survived'])
titanic_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,1,1,0,0,8,2
1,1,0,4,1,0,4,2,0
2,3,0,2,0,0,1,8,2
3,1,0,4,1,0,4,2,2
4,3,1,4,0,0,1,8,2


In [94]:

X_train = titanic_train.values
X_test = titanic_test.values

In [95]:
clf = svm.SVC()
model = clf.fit(X_train, y)

In [96]:
survived = model.predict(X_test)

In [97]:
save_id = save_id.values
results =  {'PassengerId': save_id,
            'Survived': survived}

In [98]:
results = pd.DataFrame(results)

In [99]:
results.to_csv("result.csv", encoding='utf-8', index=False)